### 泰坦尼克乘客生存预测

### 1、数据探索
** info（）了解数据表的基本情况：行数，列数，每列的数据类型，数据完整度

** describe（）了解数据表的统计情况：总数，平均值，标准差，最小值，最大值等

** describe（include = ['O']）里面是大写字母O，查看字符串类型（非数字）的整体情况

** 使用head查看前几行数据

** 使用tail查看后几行数据

In [36]:
import pandas as pd
from IPython.display import display

In [39]:
# 数据加载
train_data = pd.read_csv('E:/Jupyter/算法/data/19.Titanic_Data-master/train.csv')
test_data = pd.read_csv('E:/Jupyter/算法/data/19.Titanic_Data-master/test.csv')
display(train_data.head())
display(test_data.tail())

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S
417,1309,3,"Peter, Master. Michael J",male,NaN,1,1,2668,22.3583,NaN,C


In [57]:
# 数据探索
display(train_data.info())
print('-'*100)
display(train_data.describe())  # 因为Age存在NaN，所以百分比计算中警告有无效值
print('-'*100)
display(train_data.describe(include =['O']))  # 非数字字段

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


None

----------------------------------------------------------------------------------------------------


F:\Program Files (x86)\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,NaN,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,NaN,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,NaN,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


----------------------------------------------------------------------------------------------------


,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,204,889
unique,891,2,681,147,3
top,"Harder, Mr. George Achilles",male,CA. 2343,B96 B98,S
freq,1,577,7,4,644


### 2、数据清洗
* 通过info可以看出训练集中Age、Cabin（船舱）、Embarked（登录港口）有缺失值，测试集中Age、Cabin有缺失值
* Age，Embarked有少量缺失值可补齐，但Cabin大量缺失值无法补齐

In [74]:
# 使用平均年龄来填充年龄中的nan值
train_data['Age'].fillna(train_data['Age'].mean(),inplace = True)
test_data['Age'].fillna(train_data['Age'].mean(),inplace = True)
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [77]:
# Cabin有大量的缺失值，无法补齐。
# Embarked为登录港口，有少量缺失值，可以补齐
# 先观察Embarked字段的取值
display(train_data['Embarked'].value_counts())

# 三个登录港口中，S港口人数最多，把其余缺失的Embarked数值均设置为S
train_data['Embarked'].fillna('S',inplace = True)
test_data['Embarked'].fillna('S',inplace = True)

train_data.info()

S    646
C    168
Q     77
Name: Embarked, dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       891 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


### 3、特征选择
* 特征选择时分类器的关键，特征选择不同，得到的分类器也不同
* 通过数据探索我们发现，PassengerId（乘客编号）、Name（姓名），对分类无作用，可以排除；Cabin（船舱）缺失值太多，排除；Ticket杂乱无章，排除。
* 其余字段：Pclass（船票等级）、Sex（性别）、Age（年龄）、SibSp（兄妹、配偶数）、Parch（父母、子女数）、Fare（船票价格）、Embarked（登录港口），可能会与乘客的生存预测分类相关。

In [82]:
# 把其余字段作为特征放到特征向量features里
features = ['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']
train_features = train_data[features]
train_labels = train_data['Survived']
train_features.head(10)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.000000,1,0,7.2500,S
1,1,female,38.000000,1,0,71.2833,C
2,3,female,26.000000,0,0,7.9250,S
3,1,female,35.000000,1,0,53.1000,S
4,3,male,35.000000,0,0,8.0500,S
5,3,male,29.699118,0,0,8.4583,Q
6,1,male,54.000000,0,0,51.8625,S
7,3,male,2.000000,3,1,21.0750,S
8,3,female,27.000000,0,2,11.1333,S
9,2,female,14.000000,1,0,30.0708,C


In [89]:
# 可以看到特征值sex和embarked是字符串，不方便后续运算，需要转成数值类型
# sex字段可以变成两个字段 sex = male 和 sex = female，用数值0或1来表示
# embarked有S\C\Q三种可能，也改成Embarked=S、Embarked=C、Embarked=Q三个字段，用数值0或1来表示

#train_features['Sex'] = train_features['Sex'].map({'male':0,'female':1}) #将sex = male转为0，sex = female转为1

# 第二种方法
# 使用sklearn特征选择的DictVectorizer类，可以处理符号化现象，将符号转化成数字0、1进行表示
from sklearn.feature_extraction import DictVectorizer
dvec = DictVectorizer(sparse = False)  #sparse=False意思是不产生稀疏矩阵

# fit_transform函数，可以将特征向量转化为特征值矩阵
train_features = dvec.fit_transform(train_features.to_dict(orient = 'record'))

# devc在转化后的特征属性通过feature_names_查看
print(dvec.feature_names_)

['Age', 'Embarked=C', 'Embarked=Q', 'Embarked=S', 'Fare', 'Parch', 'Pclass', 'Sex=female', 'Sex=male', 'SibSp']


### 4、决策树模型
** 使用ID3算法，即在创建DecisionTreeClassifier时，设置criterion = 'entropy'

** 然后使用fit进行训练，将特征矩阵和分类标识结果作为参数传入，得到决策树分类器

In [97]:
from sklearn.tree import DecisionTreeClassifier
# 构造ID3决策树
clf = DecisionTreeClassifier(criterion = 'entropy')
# 决策树训练
clf = clf.fit(train_features,train_labels)
clf

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

### 5、模型预测&评估
** 在预测中，首先需要得到测试集的特征值矩阵，然后训练好的决策树clf进行预测，得到预测结果pred_labels

In [111]:
test_features = test_data[features]
test_features = dvec.transform(test_features.to_dict(orient = 'record'))
print(dvec.feature_names_)
# 代入测试集的特征值矩阵到决策树，得出预测结果
pred_labels = clf.predict(test_features)
#pred_labels

['Age', 'Embarked=C', 'Embarked=Q', 'Embarked=S', 'Fare', 'Parch', 'Pclass', 'Sex=female', 'Sex=male', 'SibSp']


In [123]:
# 由于test.csv中没有实际的Survived结果，预测出来的pred_labels无法与之作比较评判准确率
# 只能使用训练集中的数据进行模型评估

# 得到决策树准确率
acc_decision_tree = clf.score(train_features,train_labels)
print('score准确率为：%.4f' %acc_decision_tree)
#用训练集做训练，在用训练集自身做准确率评估自然高，得出的准确率不能代表决策树分类器的准确率

score准确率为：0.9820


### 6、K折交叉验证方法：对于不知道测试集实际结果的，使用该方法才能知道模型的准确率
** 原理：拿出大部分样本进行训练，少量的用于分类器的验证。就是做K次交叉验证，每次选取K分之一的数据作为验证，其余作为训练，轮流K次，取平均值

** 1.将数据集平均分割成K个等分

** 2.使用1份数据作为测试数据，其余作为训练数据

** 3.计算测试准确率

** 4.使用不同的测试集，重复2和3步骤

** sklearn的 model_selection模型提供cross_val_score函数，函数中的参数cv代表对原始数据划分成多少份，也就是我们的K值，一般建议K值取10，因此可以设置CV = 10

In [146]:
from sklearn.model_selection import cross_val_score
# #cv：选择每次测试折数  accuracy：评价指标是准确度,可以省略使用默认值
cvs = cross_val_score(clf,train_features,train_labels,cv = 10,scoring='accuracy')
cvs

array([ 0.7       ,  0.78888889,  0.68539326,  0.80898876,  0.84269663,
        0.76404494,  0.82022472,  0.74157303,  0.83146067,  0.85227273])

In [148]:
import numpy as np
print('准确率为%.4f' % np.mean(cvs))

准确率为0.7836


## 总结：
### 1、特征选择时分类模型好坏的关键，选择什么特征，对应的特征值矩阵，决定了分类模型的好坏，通常情况下，特征值不都是数值类型，可以使用DictVectorizer类进行转化
### 2、模型准确率需要考虑是否有测试集的实际结果可以做对比，当测试集没有真实结果可以对比时，需要使用K折交叉验证cross_val_score
### 3、DictVectorizer类中提供fit_transform函数和transform函数，区别是：
* fit_transform()的作用是先拟合数据，然后将其转化为标准形式，一般应用在训练集中
* transform()是通过找中心和缩放等实现标准化，一般用在训练集中